In [2]:
import os
from pprint import pprint
from openai import OpenAI
import bson
from dotenv import load_dotenv
import pymongo

# Load config from a .env file:
load_dotenv("config.env",verbose=True)
MONGODB_URI = os.environ["MONGO_URI"]

# Connect to your MongoDB cluster:
client = pymongo.MongoClient(MONGODB_URI)

# Get a reference to the "sample_mflix" database:
db = client["cluster0"]

# Get a reference to the "movies" collection:
collection  = db["40klore"]


In [3]:
def  get_embeddings(text):
    key = os.environ['OPENAI_KEY']
    client = OpenAI(api_key=key)
    return client.embeddings.create(input = [text], model="text-embedding-3-small").data[0].embedding


def vector_search(query, num_results=10):
    query_emb = get_embeddings(query)
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "path": "embedding",
                "queryVector": query_emb,
                "numCandidates": 200,
                "limit": num_results
            }
        }
    ]
    return collection.aggregate(pipeline)

In [4]:

query = "Who led the 13th black crusade?"


results = vector_search(query)

for doc in results:
   print(" * {title}, {content}, {url}".format(
         title=doc["title"],
         content=doc["content"],
         url=doc["url"],
   ))

 * 13th Black Crusade (Original) , The 13th Black Crusade began as the in-game background to the 2003 worldwide Warhammer 40,000 Eye of Terror Campaign. At the time, it resulted in a minor victory for the Forces of Chaos. Its temporary conclusion provided the Warmaster of Chaos Undivided, Abaddon the Despoiler of the Black Legion, and his Chaotic allies the territorial foothold they had long sought within the Cadian Gate, though the servants of the Dark Gods suffered grievous losses to their naval warfleet.
For several solar months after the pause in major combat operations, the surface of the Fortress World of Cadia was largely controlled by the Forces of Chaos, while the skies above Cadia were almost entirely ruled by Lord Admiral Quarren and the Gothic and Solar Battlefleet Groups as well as fleet elements from several Chapters of Space Marines. Thus, for a brief time, it seemed the ultimate result of the 13th Black Crusade was a stalemate between the forces of the Emperor and the R